In [3]:
import pandas as pd

from langchain.prompts import ChatPromptTemplate

import os
import pandas as pd

from langchain_openai import ChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_core.output_parsers import StrOutputParser
import json

from langchain_core.runnables import RunnableSequence
import os

In [4]:
NAMESPACE = 'TESLA_MODEL3'
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="my-small_mechanic"

In [5]:
df = pd.read_parquet(f'../../pdf_context/embedding/{NAMESPACE}_embeddings.parquet')
df = df[['doc_id', 'doc_contents']].drop_duplicates()

In [61]:
system_prompt = '당신은 고급 질문-답변 생성 AI입니다. 주어진 문서를 분석하고, 그 내용에 기반하여 관련성 높고 다양한 질문-답변 쌍을 생성해야 합니다.'
human_prompt = '''
다음 지침을 따라주세요: 

1. 주어진 문서를 주의 깊게 읽고 핵심 주제와 중요 정보를 파악하세요. 

2. 문서의 길이와 복잡성에 따라 최소 3개 ~ 최대 10개의 질문-답변 쌍을 생성하세요. 

3. 질문은 다음 유형을 포함하되 이에 국한되지 않습니다: - 사실 확인 질문 - 개념 이해 질문 - 분석적 질문 - 응용 질문 

4. 답변은 간결하면서도 완전해야 하며, 문서의 정보만을 사용하여 작성하세요. 

5. 질문과 답변을 다음 형식의 JSON 문자열로 출력하세요:
[
  {{"question": "질문 1", "answer": "답변 1"}},
  {{"question": "질문 2", "answer": "답변 2"}},
  ...
]

6. 생성된 질문-답변 쌍이 문서의 주요 내용을 포괄적으로 다루는지 확인하세요.

주의: 반드시 위의 형식을 정확히 따라 JSON 문자열을 출력해야 합니다. 번호를 매기거나 추가 설명을 붙이지 마세요.

이제 제공된 문서를 기반으로 질문-답변 쌍을 생성해 주세요.

<문서>
{context}
</문서>
'''

llm = ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'], 
                        temperature=0.2, 
                        model="gpt-3.5-turbo") 
# qa_reduce_template = ChatPromptTemplate.from_template(self.qa_reduce_prompt)
qa_creation_template = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", human_prompt)
    ])
llm = ChatOpenAI(api_key=os.environ['OPENAI_API_KEY'], 
                 temperature=0.2, 
                 model="gpt-3.5-turbo")

qa_creation_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", human_prompt)
])

# 단일 문서 처리를 위한 체인을 RunnableSequence로 정의
single_doc_chain = RunnableSequence(
    qa_creation_template,
    llm
)

# 메인 실행 코드
documents = df['doc_contents'].tolist()  # 예: df는 pandas DataFrame

# 입력 준비
inputs = [{"context": doc} for doc in documents]


In [63]:

# 병렬 처리 실행
results = single_doc_chain.batch(inputs, {"max_concurrency": 5})


In [67]:
import pickle

In [69]:
with open('qaset.pickle', 'wb') as f:
    pickle.dump(results, f)

In [76]:
from langchain_core.messages import AIMessage
qa_list = []
# 결과 처리
for i, result in enumerate(results):
    if isinstance(result, AIMessage):
        content = result.content
    else:
        content = result  # 이미 문자열인 경우

    try:
        # JSON 문자열을 Python 객체로 파싱
        qa_pairs = json.loads(content)
        qa_list.append(qa_pairs)
        
    except json.JSONDecodeError:
        print(f"Error parsing JSON for document {i}")
        print("Raw content:")
        qa_list.append([])
        print(content)
    

print(f"Processed {len(results)} documents successfully.")

Error parsing JSON for document 322
Raw content:
[
    {"question": "ISOFIX/i-Size 고정 바는 어디에 위치하고 있나요?", "answer": "Model 3의 후면 바깥쪽 착석 위치에 위치하고 있습니다."},
    {"question": "어린이 보호 장치를 어떻게 선택해야 하나요?", "answer": "어린이의 체중, 신장 및 연령에 적합한지 확인해야 합니다."},
    {"question": "어린이 보호 장치를 장착할 때 주의해야 할 점은 무엇인가요?", "answer": "어린이에게 너무 두꺼운 옷을 입히지 말아야 하고, 어린이와 장치 사이에 어떠한 물건도 놓아 두면 안 되며, 매번 주행할 때마다 하니스를 모든 어린이에게 맞게 조정해야 합니다."},
    {"question": "ISOFIX/i-Size 어린이 보호 장치를 장착하는 방법은 무엇인가요?", "answer": "어린이 보호 장치의 제조사가 제공한 지침을 주의하여 읽고 따라야 합니다. 좌석 고정 바로 밀어 "딸깍" 소리가 나도록 제자리에 고정하는 방법을 설명합니다."},
    {"question": "어린이 보호 장치가 제대로 장착되었는지 확인하는 방법은 무엇인가요?", "answer": "어린이 보호 장치를 좌우로 돌려보거나 좌석과 반대쪽으로 당겨보는 등의 테스트를 해야 합니다. 만약 장치가 돌거나 좌석과 멀어진다면 다시 장착해야 합니다."},
    {"question": "ISOFIX/i-Size 고정 장치를 사용할 때 주의해야 할 무게 제한은 얼마인가요?", "answer": "어린이와 어린이 보호 장치의 합산 무게가 33kg을 초과할 경우 사용하면 안 됩니다."},
    {"question": "ISOFIX/i-Size 어린이 보호 장치를 장착할 때 중요한 점은 무엇인가요?", "answer": "어린이 보호 장치의 두 래치 모두가 좌석의 고정 바에 완전히 체결되어 있어야 합니다."}
]
Processed 4

In [79]:
df['qaset'] = qa_list

In [96]:
df2 = df.copy()
df2 = df2.dropna()

In [97]:
df2.isna().sum()

doc_id          0
doc_contents    0
qaset           0
dtype: int64

In [98]:
df2 = df2.explode('qaset')
df2 = df2.dropna()

In [103]:
df2[['question', 'answer']] = df2['qaset'].apply(lambda x: pd.Series([x['question'], x['answer']]))

/var/folders/49/sp_nrcwj4cl4wq36jvgbh38h0000gn/T/ipykernel_35370/1084042290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['question', 'answer']] = df2['qaset'].apply(lambda x: pd.Series([x['question'], x['answer']]))
/var/folders/49/sp_nrcwj4cl4wq36jvgbh38h0000gn/T/ipykernel_35370/1084042290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2[['question', 'answer']] = df2['qaset'].apply(lambda x: pd.Series([x['question'], x['answer']]))


In [106]:
df2.to_parquet('./TESLA_MODEL3_QASET.parquet', index=False)
df2.to_csv('./TESLA_MODEL3_QASET.csv', index=False)

In [15]:
df = pd.read_parquet('./TESLA_MODEL3_QASET.parquet')


# Hit Rate

In [16]:
import sys
import os
sys.path.append('../')

In [17]:
from reranker import Reranker
from car_manual_bot_ipy import car_manual_generator
from transformers import pipeline
import numpy as np

In [18]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="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"
os.environ["LANGCHAIN_PROJECT"]="my-small_mechanic"

milvus_host = 'localhost'#os.environ["MILVUS_HOST"]#'localhost'#
milvus_port = os.environ["MILVUS_PORT"]

DB_COLLECTION_NAME = "HYUNDAI_CAR_MANUAL"
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
NAMESPACE_TYPE = ["IONIQ5_2024", "SANTAFE_MX5_2023", "SONATA_DN8_2024", 'TESLA_MODEL3']
NAMESPACE = NAMESPACE_TYPE[3]

CONTEXT_PATH = '/Users/yj/Kim/1.work/SKR/8.GenAI/my-small-mechanic/pdf_context'

sparse_params = {"drop_ratio_search": 0.01}
dense_params = {"ef": 100}

In [19]:

DEVICE='mps'
RERANKER = pipeline("text-classification", model="Dongjin-kr/ko-reranker", device=DEVICE)
generator = car_manual_generator(OPENAI_API_KEY, NAMESPACE, milvus_host, milvus_port, DB_COLLECTION_NAME, 5, llm_model="gpt-3.5-turbo", rrk_weight=(0.4,0.6),
                 score_filter=False, threshold=0.01, drop_duplicates=True, pandas_llm_model="gpt-3.5-turbo", reduce_model="gpt-3.5-turbo",
                 map_text_model="gpt-3.5-turbo", context_path=CONTEXT_PATH, reranker=RERANKER)

In [20]:
from concurrent.futures import ThreadPoolExecutor
import threading


df_sample = df.groupby('doc_id').head(4).copy()
print(df_sample.shape)

# progress_id = 0
# lock = threading.Lock()

def process_question(generator, question):
    global progress_id
    context, score = generator.get_context(question)
    doc_ids = list(map(lambda x: x['parent_doc_id'], context))
    # with lock:
    #     progress_id += 1
    #     print(progress_id)
    return doc_ids

def parallel_process(df, generator, num_threads=None):
    if num_threads is None:
        num_threads = threading.active_count() * 2  # 기본 스레드 수의 2배

    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        results = list(
            executor.map(lambda q: process_question(generator, q), df['question'].values))
    
    return results
# 병렬 처리 실행
doc_ids_list = parallel_process(df_sample, generator)


(1645, 5)


2024-07-09 12:20:13,800 [INFO] Start Rerank
2024-07-09 12:20:13,803 [INFO] Start Rerank
2024-07-09 12:20:13,805 [INFO] Start Rerank
2024-07-09 12:20:13,808 [INFO] Start Rerank
2024-07-09 12:20:13,814 [INFO] Start Rerank
2024-07-09 12:20:13,821 [INFO] Start Rerank
2024-07-09 12:20:13,823 [INFO] Start Rerank
2024-07-09 12:20:13,825 [INFO] Start Rerank
2024-07-09 12:20:13,829 [INFO] Start Rerank
2024-07-09 12:20:13,831 [INFO] Start Rerank
2024-07-09 12:20:13,833 [INFO] Start Rerank
2024-07-09 12:20:13,836 [INFO] Start Rerank
2024-07-09 12:20:13,838 [INFO] Start Rerank
2024-07-09 12:20:13,840 [INFO] Start Rerank
/Users/yj/.pyenv/versions/3.9.11/envs/myMechanic_py39/lib/python3.9/site-packages/transformers/models/xlm_roberta/modeling_xlm_roberta.py:1578: UserWarning: cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:425.)
  incremental_indices = (torch.cumsum(mask, 

In [23]:
padded_data = [sublist + [np.nan] * (5 - len(sublist)) for sublist in doc_ids_list]

# DataFrame 생성
retreiver_orders = pd.DataFrame(padded_data, columns=[f'retreiver_{i+1}' for i in range(5)])

In [27]:
df_sample = df_sample.reset_index(drop=True)

In [28]:
df_sample[retreiver_orders.columns] = retreiver_orders

In [29]:
def check_match(row, target_column='doc_id', compare_columns = ['retreiver_1']):
    target_value = row[target_column]
    other_values = set(row[compare_columns])
    return target_value in other_values

In [34]:
df_sample = df_sample.drop_duplicates('question')

In [30]:
df_sample['top1'] = df_sample.apply(lambda x: check_match(x, 'doc_id', ['retreiver_1']), axis=1)
df_sample['top2'] = df_sample.apply(lambda x: check_match(x, 'doc_id', ['retreiver_1', 'retreiver_2']), axis=1)
df_sample['top3'] = df_sample.apply(lambda x: check_match(x, 'doc_id', ['retreiver_1', 'retreiver_2', 'retreiver_3']), axis=1)
df_sample['top4'] = df_sample.apply(lambda x: check_match(x, 'doc_id', ['retreiver_1', 'retreiver_2', 'retreiver_3', 'retreiver_4']), axis=1)
df_sample['top5'] = df_sample.apply(lambda x: check_match(x, 'doc_id', ['retreiver_1', 'retreiver_2', 'retreiver_3', 'retreiver_4', 'retreiver_5']), axis=1)

0.3 vs 0.7

In [35]:
print(df_sample['top1'].sum()/df_sample.shape[0])
print(df_sample['top2'].sum()/df_sample.shape[0])
print(df_sample['top3'].sum()/df_sample.shape[0])
print(df_sample['top4'].sum()/df_sample.shape[0])
print(df_sample['top5'].sum()/df_sample.shape[0])

0.7998768472906403
0.8873152709359606
0.9076354679802956
0.9174876847290641
0.9193349753694581


In [32]:
df_sample.to_parquet('./qa_predict_new.parquet', index=False)

In [44]:
df_sample.loc[1638]['answer']

'터치스크린에서 충전 화면을 표시하여 Model 3이(가) 현재 충전 중인지 확인하고, 후면 트렁크를 열고 충전 포트의 해제 케이블을 아래로 당겨 충전 케이블 래치를 해제한 후, 충전 케이블을 충전 포트에서 당기면 됩니다.'

In [67]:
df_sample[df_sample['top1']][['doc_id', 'doc_contents', 'question', 'answer']].sample(10)

,doc_id,doc_contents,question,answer
91,22,"[최근 항목 및 즐겨찾기]\n최근 항목 및 즐겨찾기\n\n대부분의 최신 콘텐츠, 최...",여러 즐겨찾기를 한 번에 제거하는 방법은 무엇인가요?,미디어 플레이어를 확장하여 해당 유형의 콘텐츠에 대한 모든 즐겨찾기를 표시하고 X를...
1327,335,[에어백의 유형]\n에어백의 유형\n\nModel 3 다음과 같은 유형의 에어백이 ...,에어백의 유형에는 어떤 종류가 있나요?,"에어백의 유형에는 전면 에어백, 시트 장착 측면 에어백, 커튼 에어백이 있습니다."
1423,359,[브레이크 시스템]\n유압 페이드 보상\n\n\nModel 3 에는 유압 페이드 보...,브레이크 성능 감소가 감지된 경우에는 어떻게 해야 하나요?,평상시처럼 제동하고 브레이크 페달에서 발을 떼거나 펌핑하지 말고 계속해서 밟은 상태...
746,187,[자동 긴급 제동]\n자동 긴급 제동\n\nModel 3 은(는) 감지된 물체로부터...,자동 긴급 제동은 어떤 상황에서 작동하나요?,자동 긴급 제동은 충돌을 피할 수 없는 상황으로 간주될 때 작동합니다.
38,9,[키 카드]\n키 카드\n\nTesla는 지갑에 넣고 다니기에 알맞게 고안된 두 개...,Tesla Model 3에서 키 카드를 감지했을 때 어떤 일이 벌어지나요?,"외부등이 점멸하고, 미러가 펴지거나 접히며, 경적음이 울리고 도어가 잠금 해제되거나..."
1545,390,[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오....,왜 배터리를 완전히 방전시키면 안 되는 것인가요?,배터리가 0%까지 방전되면 차량 부품이 손상될 수 있기 때문입니다.
1334,336,[팽창의 영향]\n팽창의 영향\n\n경고: 에어백이 팽창할 때 미세한 가루가 방출됩...,충돌 시 에어백 팽창 외에 어떤 상황이 발생할 수 있나요?,충돌 시 에어백 팽창 외에 도어가 잠금 해제되거나 위험 경고등이 켜지는 등 다양한 ...
1110,279,\n유지보수 정비 주기\n\n• 배터리 냉각수: 대부분의 경우 차량의 수명까지 배터...,고전압 안전을 위한 예방 조치 중 하나는 무엇인가요?,Model 3에 부착된 레이블에 쓰인 지침을 모두 읽고 따르는 것입니다.
507,126,"[저작권 및 상표권]\n저작권 및 상표권\n\n© 2012-2024 Tesla, I...","Tesla, Inc.의 상표 또는 등록 상표가 어느 국가에서 사용되고 있나요?","미국 및 기타 국가에서 Tesla, Inc.의 상표 또는 등록 상표로 사용되고 있습니다."
1427,360,\n운전자 프로필\n\n량으로 돌아와서 브레이크 페달을 밟으면 가장 최근 운전자 프...,어린이용 안전 시트를 운전석 뒤의 뒷좌석에 장착한 경우 이지 엔트리를 사용하여 운전...,"운전석 시트의 움직임으로 인해 아이의 다리에 영향을 줄 수 있으며, 부상을 유발하거..."


In [73]:
df_sample[df_sample['doc_id']==390]

,doc_id,doc_contents,qaset,question,answer,retreiver_1,retreiver_2,retreiver_3,retreiver_4,retreiver_5,top1,top2,top3,top4,top5
1545,390,[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오....,{'answer': '배터리가 0%까지 방전되면 차량 부품이 손상될 수 있기 때문입...,왜 배터리를 완전히 방전시키면 안 되는 것인가요?,배터리가 0%까지 방전되면 차량 부품이 손상될 수 있기 때문입니다.,390,392,112.0,218.0,388.0,True,True,True,True,True
1546,390,[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오....,"{'answer': 'Model 3의 배터리는 하루에 약 1%씩 방전됩니다.', '...",Model 3의 배터리는 하루에 얼마씩 방전되나요?,Model 3의 배터리는 하루에 약 1%씩 방전됩니다.,390,407,402.0,218.0,20.0,True,True,True,True,True
1547,390,[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오....,{'answer': '배터리가 0%까지 방전되면 Model 3은 저전력 소비 모드에...,배터리가 0%까지 방전되면 어떤 모드로 진입하게 되나요?,배터리가 0%까지 방전되면 Model 3은 저전력 소비 모드에 진입합니다.,390,393,146.0,5.0,112.0,True,True,True,True,True
1548,390,[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오....,{'answer': '저전력 소비 모드에서 배터리는 차내 전자 장치 및 보조 저전압...,배터리가 0%까지 방전되었을 때 저전력 소비 모드에서 어떤 기능이 중지되나요?,저전력 소비 모드에서 배터리는 차내 전자 장치 및 보조 저전압 배터리의 지원을 중지...,390,393,392.0,116.0,117.0,True,True,True,True,True


In [75]:
df

,doc_id,doc_contents,qaset,question,answer
0,0,\n실내 전자 장치\n\n휴대폰을 무선 충전기 위에 올려 놓으면 차량 전원이 켜 있...,{'answer': '차량 전원이 켜 있을 때(터치스크린이 켜 있고 운전자가 차량 ...,무선 충전기 위에 휴대폰을 올려 놓으면 언제 휴대폰이 충전되나요?,차량 전원이 켜 있을 때(터치스크린이 켜 있고 운전자가 차량 내부에 있을 경우) 언...
1,0,\n실내 전자 장치\n\n휴대폰을 무선 충전기 위에 올려 놓으면 차량 전원이 켜 있...,"{'answer': '실내 온도 유지, 애견 모드, 캠핑 모드 또는 감시 모드와 같...",휴대폰이 충전되지 않는 경우는 어떤 경우인가요?,"실내 온도 유지, 애견 모드, 캠핑 모드 또는 감시 모드와 같은 기능이 켜져 있지 ..."
2,0,\n실내 전자 장치\n\n휴대폰을 무선 충전기 위에 올려 놓으면 차량 전원이 켜 있...,{'answer': '휴대폰 케이스가 너무 두껍거나 금속 재질인 경우에는 스마트폰 ...,휴대폰 무선 충전기가 작동하지 않을 수 있는 경우는 어떤 경우인가요?,휴대폰 케이스가 너무 두껍거나 금속 재질인 경우에는 스마트폰 무선 충전기가 작동하지...
3,0,\n실내 전자 장치\n\n휴대폰을 무선 충전기 위에 올려 놓으면 차량 전원이 켜 있...,"{'answer': '터치스크린에는 항상 Model 3의 상태가 표시됩니다.', '...",터치스크린에 어떤 정보가 항상 표시되나요?,터치스크린에는 항상 Model 3의 상태가 표시됩니다.
4,0,\n실내 전자 장치\n\n휴대폰을 무선 충전기 위에 올려 놓으면 차량 전원이 켜 있...,"{'answer': 'Model 3이 주차 상태일 때는 주행 모드, 예상 주행 가능...",Model 3이 주차 상태일 때 표시되는 정보는 무엇인가요?,"Model 3이 주차 상태일 때는 주행 모드, 예상 주행 가능 거리, 차량의 조감 ..."
...,...,...,...,...,...
2751,414,[타사 충전기]\n타사 충전기\n\n판매 지역 및 차량 구성에 따라 모든 Tesla...,{'answer': 'CCS 커넥터가 있는 타사 스테이션에서 충전하려면 어댑터가 필...,CCS 커넥터가 있는 타사 스테이션에서 충전하려면 어댑터가 필요한가요?,CCS 커넥터가 있는 타사 스테이션에서 충전하려면 어댑터가 필수적으로 필요합니다.
2752,414,[타사 충전기]\n타사 충전기\n\n판매 지역 및 차량 구성에 따라 모든 Tesla...,{'answer': '차량이 CCS와 통신할 수 있는지 확인하기 위해서는 컨트롤 >...,차량이 CCS와 통신할 수 있는지 확인하기 위해 어디로 이동해야 하나요?,차량이 CCS와 통신할 수 있는지 확인하기 위해서는 컨트롤 > 소프트웨어 > 추가 ...
2753,414,[타사 충전기]\n타사 충전기\n\n판매 지역 및 차량 구성에 따라 모든 Tesla...,"{'answer': 'Model 3이 CCS 통신 프로토콜을 지원하지 않는 경우, ...",Model 3이 CCS 통신 프로토콜을 지원하지 않는 경우 어떤 스테이션에서 충전이...,"Model 3이 CCS 통신 프로토콜을 지원하지 않는 경우, 타사 NACS 또는 C..."
2754,414,[타사 충전기]\n타사 충전기\n\n판매 지역 및 차량 구성에 따라 모든 Tesla...,{'answer': 'CCS 충전기를 사용할 수 있는 차량은 Model 3 CCS ...,CCS 충전기를 사용할 수 있는 차량은 어떤 기능을 제공하나요?,CCS 충전기를 사용할 수 있는 차량은 Model 3 CCS 통신 프로토콜을 지원하...


In [74]:
df_sample[df_sample['doc_id']==390][['question', 'answer']]

,question,answer
1545,왜 배터리를 완전히 방전시키면 안 되는 것인가요?,배터리가 0%까지 방전되면 차량 부품이 손상될 수 있기 때문입니다.
1546,Model 3의 배터리는 하루에 얼마씩 방전되나요?,Model 3의 배터리는 하루에 약 1%씩 방전됩니다.
1547,배터리가 0%까지 방전되면 어떤 모드로 진입하게 되나요?,배터리가 0%까지 방전되면 Model 3은 저전력 소비 모드에 진입합니다.
1548,배터리가 0%까지 방전되었을 때 저전력 소비 모드에서 어떤 기능이 중지되나요?,저전력 소비 모드에서 배터리는 차내 전자 장치 및 보조 저전압 배터리의 지원을 중지...


In [71]:
df_sample[df_sample['doc_id']==390]['doc_contents'].values[0]

'[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오.\n\nModel 3을(를) 운전하지 않더라도 차량 전장에 전원을 공급하기 위해 배터리는 매우 천천히 방전됩니다. 배터리 방전률은 환경 요인(예: 추운 날씨), 차량 구성 및 터치스크린에서 선택한 설정에 따라 달라질 수 있지만 하루에 약 1%씩 방전됩니다. Model 3을(를) 오랜 기간 동안 전원에 연결하지 못한 상태로 두어야 하는 상황이 발생할 수 있습니다(공항에 두고 여행하는 등). 이러한 1%를 유념해 배터리 잔량을 충분히 남겨두십시오. 예를 들면, 2주(14일) 동안 배터리는 약 14% \n\n방전될 수 있습니다.\n\n배터리가 0%까지 방전되면 차량 부품이 손상될 수 있습니다. 완전히 방전되는 것을 방지하기 위해 Model 3은(는) 표시된 충전 레벨이 약 0%까지 떨어지면 저전력 소비 모드에 진입합니다. 이 모드에서 배터리는 차내 전자 장치 및 보조 저전압 배터리의 지원을 중지합니다. 이 저전력 소비 모드가 활성화되면 즉시 Model 3에 충전 케이블을 연결하여 점프 시동 및 저전압 배터리 교체와 같은 상황이 발생하지 않도록 하십시오.\n\n참고: Model 3이(가) 응답하지 않고 잠금 해제, 도어 개방 또는 충전이 되지 않는 경우 저전압 배터리가 방전되었을 수 있습니다. 이러한 상황에서는 저전압 배터리 점프 시동을 시도하세요(점프 시동 페이지의 194 참조). 차량이 여전히 반응하지 않는 경우 모바일 앱을 사용하여 정비 예약을 하세요.\n\n온도 제한\n\n\n오랫동안 더 좋은 성능을 발휘할 수 있도록 Model 3을(를) 60°C 초과 또는 -30°C 미만의 외기 온도에 연속해서 24시간 넘게 노출시키지 마십시오.'

In [72]:
print('[배터리 관리]\n배터리 관리\n\n절대로 배터리가 완전히 방전되게 하지 마십시오.\n\nModel 3을(를) 운전하지 않더라도 차량 전장에 전원을 공급하기 위해 배터리는 매우 천천히 방전됩니다. 배터리 방전률은 환경 요인(예: 추운 날씨), 차량 구성 및 터치스크린에서 선택한 설정에 따라 달라질 수 있지만 하루에 약 1%씩 방전됩니다. Model 3을(를) 오랜 기간 동안 전원에 연결하지 못한 상태로 두어야 하는 상황이 발생할 수 있습니다(공항에 두고 여행하는 등). 이러한 1%를 유념해 배터리 잔량을 충분히 남겨두십시오. 예를 들면, 2주(14일) 동안 배터리는 약 14% \n\n방전될 수 있습니다.\n\n배터리가 0%까지 방전되면 차량 부품이 손상될 수 있습니다. 완전히 방전되는 것을 방지하기 위해 Model 3은(는) 표시된 충전 레벨이 약 0%까지 떨어지면 저전력 소비 모드에 진입합니다. 이 모드에서 배터리는 차내 전자 장치 및 보조 저전압 배터리의 지원을 중지합니다. 이 저전력 소비 모드가 활성화되면 즉시 Model 3에 충전 케이블을 연결하여 점프 시동 및 저전압 배터리 교체와 같은 상황이 발생하지 않도록 하십시오.\n\n참고: Model 3이(가) 응답하지 않고 잠금 해제, 도어 개방 또는 충전이 되지 않는 경우 저전압 배터리가 방전되었을 수 있습니다. 이러한 상황에서는 저전압 배터리 점프 시동을 시도하세요(점프 시동 페이지의 194 참조). 차량이 여전히 반응하지 않는 경우 모바일 앱을 사용하여 정비 예약을 하세요.\n\n온도 제한\n\n\n오랫동안 더 좋은 성능을 발휘할 수 있도록 Model 3을(를) 60°C 초과 또는 -30°C 미만의 외기 온도에 연속해서 24시간 넘게 노출시키지 마십시오.')

[배터리 관리]
배터리 관리

절대로 배터리가 완전히 방전되게 하지 마십시오.

Model 3을(를) 운전하지 않더라도 차량 전장에 전원을 공급하기 위해 배터리는 매우 천천히 방전됩니다. 배터리 방전률은 환경 요인(예: 추운 날씨), 차량 구성 및 터치스크린에서 선택한 설정에 따라 달라질 수 있지만 하루에 약 1%씩 방전됩니다. Model 3을(를) 오랜 기간 동안 전원에 연결하지 못한 상태로 두어야 하는 상황이 발생할 수 있습니다(공항에 두고 여행하는 등). 이러한 1%를 유념해 배터리 잔량을 충분히 남겨두십시오. 예를 들면, 2주(14일) 동안 배터리는 약 14% 

방전될 수 있습니다.

배터리가 0%까지 방전되면 차량 부품이 손상될 수 있습니다. 완전히 방전되는 것을 방지하기 위해 Model 3은(는) 표시된 충전 레벨이 약 0%까지 떨어지면 저전력 소비 모드에 진입합니다. 이 모드에서 배터리는 차내 전자 장치 및 보조 저전압 배터리의 지원을 중지합니다. 이 저전력 소비 모드가 활성화되면 즉시 Model 3에 충전 케이블을 연결하여 점프 시동 및 저전압 배터리 교체와 같은 상황이 발생하지 않도록 하십시오.

참고: Model 3이(가) 응답하지 않고 잠금 해제, 도어 개방 또는 충전이 되지 않는 경우 저전압 배터리가 방전되었을 수 있습니다. 이러한 상황에서는 저전압 배터리 점프 시동을 시도하세요(점프 시동 페이지의 194 참조). 차량이 여전히 반응하지 않는 경우 모바일 앱을 사용하여 정비 예약을 하세요.

온도 제한


오랫동안 더 좋은 성능을 발휘할 수 있도록 Model 3을(를) 60°C 초과 또는 -30°C 미만의 외기 온도에 연속해서 24시간 넘게 노출시키지 마십시오.


In [49]:
print(df_sample[df_sample['top1'] & df_sample['question'].str.contains('충전')][['doc_contents', 'question', 'answer']].tail(50)['doc_contents'].values[3])

[플러그 연결]
플러그 연결

필요한 경우 터치스크린을 사용하여 충전 제한 및 충전 전류를 변경합니다(충전 설정 페이지의 153 참조).

공공 충전 시설에서 충전하려면 차량의 충전 포트에 적합한 어댑터를 꽂은 후, 어댑터에 충전 시설의 충전 커넥터를 연결합니다. 각 판매 지역에서 가장 일반적으로 사용되는 어댑터가 제공됩니다. 사용하는 충전 장비에 따라 충전 장비의 조작부로 충전을 시작하거나 중지해야 할 수도 있습니다.

모바일 커넥터를 사용하는 경우 전원 콘센트에 먼저 삽입한 후 Model 3와 연결하십시오.

커넥터를 충전 포트에 정렬한 후 끝까지 삽입하십시오. 커넥터를 올바르게 삽입하면 Model 3가 다음 상태가 된 이후 자동으로 충전이 시작됩니다.

• 래치가 체결되어 커넥터가 제 위치에 연결된 경우

• 주차로 변속(다른 주행 모드에 있었던 경우)

• 필요에 따라 배터리를 가열 또는 냉각한 후 배터리가 가열 또는 냉각이 필요한 경우 충전 시작 시점이 다소 지연되는 것을 볼 수 있습니다.


In [ ]:
generator = car_manual_generator(OPENAI_API_KEY, NAMESPACE, milvus_host, milvus_port, DB_COLLECTION_NAME, 5, llm_model="gpt-3.5-turbo", rrk_weight=(0.4,0.6),
                 score_filter=False, threshold=0.01, drop_duplicates=True, pandas_llm_model="gpt-3.5-turbo", reduce_model="gpt-3.5-turbo",
                 map_text_model="gpt-3.5-turbo", context_path=CONTEXT_PATH, reranker=RERANKER)

In [58]:
df_sample[~df_sample['top3']][['question', 'doc_id', 'retreiver_1', 'retreiver_2', 'retreiver_3']]

,question,doc_id,retreiver_1,retreiver_2,retreiver_3
1,휴대폰이 충전되지 않는 경우는 어떤 경우인가요?,0,100,69,75
2,휴대폰 무선 충전기가 작동하지 않을 수 있는 경우는 어떤 경우인가요?,0,100,16,84
6,미디어를 재생할 때 사용할 수 있는 명령어는 어떤 것들이 있나요?,2,26,20,19
8,앱 및 설정을 탐색할 때 사용할 수 있는 명령어는 어떤 것들이 있나요?,2,7,26,203
14,차갑거나 습한 곳에 주차할 경우 어떤 문제가 발생할 수 있나요?,4,255,172,367
...,...,...,...,...,...
1206,배터리가 충분히 방전되면 어떻게 되나요?,403,390,41,392
1211,수퍼차징을 중지하면 어떤 일이 발생하나요?,405,369,118,187
1216,배터리 용량이 20% 이하로 떨어질 때 어떤 색으로 배터리 아이콘이 바뀌는가?,407,174,5,406
1217,차량을 플러그인한 후 충전 포트 표시등이 녹색으로 점멸하는 것은 무엇을 나타내는가?,407,404,75,408


In [65]:
context, scores = generator.get_context("휴대폰이 충전되지 않는 경우는 어떤 경우인가요?	")

2024-07-09 09:58:40,115 [INFO] Start Rerank
2024-07-09 09:58:40,748 [INFO] End Rerank


In [66]:
context

[{'distance': 0.4749061167240143,
  'id': 120,
  'majorheading': '문제 해결',
  'minorheading': '문제 해결 경고',
  'minorheading_sub_id': 64,
  'parent_doc_id': 100,
  'doc_contents': '\nUMC_a005\n모바일 커넥터로 충전 불가전압이 너무 낮음 / 다른 콘센트로 시도하십시오\n\n이 경고의 의미:\n\n모바일 커넥터에서 다음 두 원인 중 하나 때문에 차량을 충전할 수 없거나 충전이 중단됩니다.\n\n• 벽면 콘센트에서 공급하는 전압을 충분히 감지하지 못합니다. 또는\n\n• 벽면 콘센트에서 공급하는 전압의 예상치 못한 강하가 감지됩니다.\n\n필요한 작업:\n\n다른 벽면 콘센트로 차량 충전을 시도합니다. 차량 충전이 시작되면 원래의 벽면 콘센트가 문제였을 가능성이 있습니다. 전기 기술자에게 연락하여 해당 콘센트에 대한 건물 배선 연결부를 점검하십시오.\n\n다른 벽면 콘센트로 충전을 시도해도 차량이 여전히 충전되지 않을 경우, 다른 장소에서 충전을 시도합니다.\n\n이 경고는 보통 외부 충전 장비 및 전력원 문제에 해당하며, 일반적으로 정비를 예약하여 해결할 수 있는 차량 문제를 나타내지 않습니다.\n\n차량 터치스크린 디스플레이의 지도를 통해 위치를 확인할 수 있는 Tesla 슈퍼차저 또는 데스티네이션 차저로 차량 충전을 시도해 볼 수도 있습니다. 자세한 내용은 지도와 내비게이션 페이지의 137을(를) 참조하십시오.\n\n모바일 커넥터 상태 표시등 및 충전 문제의 해결에 관한 자세한 내용은 해당 제품 사용자 매뉴얼를 참조하십시오.',
  'img_urls': [],
  'table_img_urls': [],
  'table_csv_urls': [],
  'embedding_contents': '문제 해결 경고\n\n\nUMC_a005\n모바일 커넥터로 충전 불가전압이 너무 낮음 / 다른 콘센트로 시도하십시오\n\n이 경고의 

In [64]:
list(map(lambda x: x['parent_doc_id'], context))

[407, 406, 6, 156, 264, 141, 158, 153, 194]

In [67]:
list(map(lambda x: x['parent_doc_id'], context))

[6, 145, 354, 404, 375, 408, 109, 168, 5]

In [94]:
list(map(lambda x: x['parent_doc_id'], context))

[407, 406, 6, 156, 264, 141, 158, 153, 194]